## Perguntas: 
1-número de fatalidades tem diminuido com os anos? *procure pelas colunas de fataliade

2-durante a noite há mais ocorrências (18:00 a 05:00)?

3-algumas regiões tem mais ocorrências que outras?

4-Verificar em quantas cidades já aconteceram ocorrências?

5-alguns operadores têm mais ocorrências que outros?

In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, hour
spark = SparkSession.builder.appName("MeuApp").getOrCreate()

In [25]:
df_ocorrencias = spark.read.option("delimiter", ";").option("header", True).csv("ocorrencia.csv")
df_ocorrencias.show(truncate=False)

+--------------------+---------------+-----------------------------------------------------------+---------------------------+------------------+------------------+----------------------+-------------+------------+--------------------------------------------------+------------------------+--------+---------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df_municipios = spark.read.option("delimiter", ",").option("header", True).csv("municipios.csv")
df_municipios.show(truncate=False)

In [ ]:
df_estados = spark.read.option("delimiter",",").option("header",True).csv("estados.csv")
df_estados.show(truncate=False, n=50)

### 1-número de fatalidades tem diminuido com os anos? *procure pelas colunas de fataliade


### 2 - Durante a noite há mais ocorrências (18:00 a 05:00)?

In [ ]:
# Definir uma função que verifica o período do dia
def periodo_dia(hora_col):
    return F.when(
        hora_col.isNull(), F.lit(None)
    ).when(
        (hora_col >= 18) | (hora_col < 5), F.lit('noite')
    ).otherwise(F.lit('dia'))

In [ ]:
# Adicionar a nova coluna 'Periodo_do_Dia' baseada na coluna 'Hora_da_Ocorrencia'
df_ocorrencias = df_ocorrencias.withColumn('Periodo_do_Dia', periodo_dia(F.hour('Hora_da_Ocorrencia')))

# Mostrar o DataFrame resultante
#df_ocorrencias = df_ocorrencias.select("Numero_da_Ocorrencia","Classificacao_da_Ocorrencia","Hora_da_Ocorrencia","Periodo_do_Dia")
df_ocorrencias.show(truncate=False)

In [ ]:
df_periodoDia = df_ocorrencias.groupBy("Periodo_do_Dia").agg(F.count(F.col("Periodo_do_Dia")).alias("Total"))
df_periodoDia.filter((F.col("Periodo_do_Dia") != "NULL")).show(truncate=False)
#df_periodoDia.show(truncate=False)

### 3 - Algumas regiões tem mais ocorrências que outras?


In [ ]:

# Realizar o join entre df_ocorrencias e df_regioes baseado na coluna "Regiao"
df_joined_regioes = df_ocorrencias.join(df_estados, "Regiao", "inner")

# Agora, realizar a agregação no DataFrame resultante do join
df_regioes =  df_joined_regioes.groupBy("Regiao").agg(F.count(F.col("Regiao")).alias("Qtd_ocorrencias")).orderBy(F.desc("Qtd_ocorrencias"))

# Exibir o resultado
df_regioes.show()


### 4 - Verificar em quantas cidades já aconteceram ocorrências?

In [ ]:

# Realizar a junção dos DataFrames
# Utilizamos o método join para combinar os dois DataFrames com base em uma chave comum.
# Aqui, estamos juntando as tabelas onde a coluna 'Municipio' no DataFrame df_ocorrencias 
# corresponde à coluna 'nome' no DataFrame df_municipios.
# O parâmetro how='inner' especifica que queremos uma junção interna, 
# ou seja, apenas as linhas com correspondência em ambas as tabelas serão mantidas.
merged_df = df_ocorrencias.join(df_municipios, df_ocorrencias.Municipio == df_municipios.nome, how='inner')

# Contar o número de municípios distintos onde ocorreram ocorrências
# Usamos select para selecionar a coluna 'Municipio' do DataFrame resultante da junção.
# O método distinct() remove duplicatas, de modo que cada município apareça apenas uma vez.
# Finalmente, count() conta o número de linhas distintas, o que nos dá o número de municípios únicos.
num_cidades_com_ocorrencias = merged_df.select("Municipio").distinct().count()

# Imprimimos o número de cidades onde ocorreram as ocorrências.
print(f"Ocorrências já aconteceram em {num_cidades_com_ocorrencias} cidades.")

### 5 - alguns operadores têm mais ocorrências que outros?

In [31]:
df_operadores = df_ocorrencias.groupBy("Operador_Padronizado").agg(F.count(F.col("Operador_Padronizado")).alias("Qtd")).orderBy(F.desc("Qtd"))

df_operadores.show(truncate=False, n=500)

+--------------------------------------------------------------------------------+----+
|Operador_Padronizado                                                            |Qtd |
+--------------------------------------------------------------------------------+----+
|AZUL                                                                            |1350|
|TAM                                                                             |912 |
|GOL                                                                             |538 |
|AZUL LINHAS AÉREAS BRASILEIRAS S.A.                                             |261 |
|AEROCLUBE DE LONDRINA                                                           |199 |
|OCEANAIR                                                                        |184 |
|LIDER TAXI AEREO S/A – AIR BRASIL                                               |183 |
|TRIP                                                                            |165 |
|null                           